In [1]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import ast
def convert_to_datetime(data_frame, key):
    try:
        data_frame[key] = pd.to_datetime(data_frame[key], format='%d/%m/%Y %H:%M:%S', utc=True)
    except:
        try:
            data_frame[key] = pd.to_datetime(data_frame[key], format='%d/%m/%Y %H:%M', utc=True)
        except:
            data_frame[key] = pd.to_datetime(data_frame[key], format='%Y-%m-%d %H:%M:%S', utc=True)

    return data_frame

def meanlist(listoflists):
    divider = len(listoflists)
    try:
        summed = listoflists.pop()
        while len(listoflists) > 0:
            summed = [sum(item) for item in zip(summed, listoflists.pop())]
        mean = []
        for number in summed:
            mean.append(number / divider)
        return mean
    except:
        return []

print("reading tweets_data")
tweets_data = pd.read_csv("raw_data/mix/Per Period/2018(03-08--03-11)_en_filtered.csv"
                          , lineterminator='\n')
data = tweets_data

data = data.sort_values(by='date')
data = data[["date", "bert", "sent_neg", "sent_neu", "sent_pos", "bot"]]

data = data[data.bot != True][["date", "bert", "sent_neg", "sent_neu", "sent_pos"]]

data = convert_to_datetime(data, "date")
data['date'] = data["date"].dt.tz_localize(None)
data = data.set_index("date")

reading tweets_data


In [7]:
aggregations = {
    'sent_neg': 'mean',
    'sent_neu': 'mean',
    'sent_pos': 'mean',
    "bert": lambda x: meanlist([ast.literal_eval(y) for y in x.values]),
    "count": 'sum'
}
data["count"] = 1
grouped_data = data.groupby(pd.Grouper(freq="10Min")).agg(aggregations)
grouped_data.head()




,sent_neg,sent_neu,sent_pos,bert,count
date,,,,,
2018-03-08 00:00:00,0.058801,0.843904,0.097301,"[0.0, 8946.38596491228, 8281.961988304094, 819...",342
2018-03-08 00:10:00,0.055884,0.868079,0.076037,"[0.0, 12626.274390243903, 8479.009146341463, 8...",328
2018-03-08 00:20:00,0.043848,0.842437,0.113711,"[0.0, 10802.33984375, 8728.6953125, 8694.56640...",256
2018-03-08 00:30:00,0.050387,0.845244,0.104365,"[0.0, 9241.545112781954, 6659.515037593985, 74...",266
2018-03-08 00:40:00,0.053265,0.825514,0.121220,"[0.0, 7262.061224489796, 5445.648979591837, 76...",245


KeyboardInterrupt: 